```markdown
# Multi-Agent Retrieval-Augmented Generation (RAG) with Vector Database
```

In [1]:
import typer
import os
from typing import Optional,List
from phi.assistant import Assistant
from phi.storage.assistant.postgres import PgAssistantStorage
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.pgvector import PgVector,SearchType
from dotenv import load_dotenv


In [2]:
load_dotenv()
os.environ['Groq_API_Key'] = os.getenv('Groq_API_Key')

In [3]:
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

In [4]:
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=PgVector(table_name="recipes", db_url=db_url, search_type=SearchType.hybrid),
)

ERROR    Failed to create engine from 'db_url': No module named 'psycopg'

ModuleNotFoundError: No module named 'psycopg'

In [ ]:
# Load the knowledge base: Comment out after first run
knowledge_base.load(recreate=True, upsert=True)

In [ ]:
storage = PgAssistantStorage(table_name="pdf_assistant",db_url=db_url)

In [ ]:
def pdf_assistant(user: str = "user"):
    run_id: Optional[str] = None

    agent = Agent(
        run_id=run_id,
        user_id=user,
        knowledge_base=knowledge_base,
        use_tools=True,
        show_tool_calls=True,
        debug_mode=True,
    )
    if run_id is None:
        run_id = agent.run_id
        print(f"Started Run: {run_id}\n")
    else:
        print(f"Continuing Run: {run_id}\n")

    while True:
        message = Prompt.ask(f"[bold] :sunglasses: {user} [/bold]")
        if message in ("exit", "bye"):
            break
        agent.print_response(message)


if __name__ == "__main__":
    typer.run(pdf_agent)